## Final Project: A Bakery in Portland, OR

### Question 1: Introduction/Business Problem

#### In this notebook, I will explore neighborhoods in Portland, Oregon in order to find a good location for a bakery/coffee shop. My audience are bakers and/or coffee shop owners that would want to start this local business. The location of the bakery is important to the audience because they want to place their shop in a neighborhood that does not have similar businesses, so there will be less competition. This audience also wants their bakery to be a neighborhood bakery where residents can easily pick up fresh bread and pastries, or grab coffee and walk at nearby parks. Therefore, the audeince wants the location to be residential, in a neighborhood with a few parks.